In [2]:
# 라이브러리 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager


from tqdm.notebook import tqdm
from tqdm import tqdm
import time
import pandas as pd 

In [3]:
# 진석님 
# 엑셀 파일 읽기
movie_df = pd.read_excel('Movie_DateBase.xlsx') 

# 인덱스 범위 지정
index_ranges = [(500, 2000+1)]  # 예시: 인덱스 범위를 원하는대로 설정

# 웹 드라이버 실행
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# 와챠피디아 URL로 이동
url = 'https://pedia.watcha.com/ko-KR'
driver.get(url)
time.sleep(1)



subset_movie_df = movie_df[500:2000+1]  # 해당 범위의 영화 데이터프레임 선택

# 해당 범위의 영화 별점 검색 및 저장
watch_aver = []  # 영화 별점을 저장할 리스트 초기화

try:
    for movie_name in tqdm(subset_movie_df['Name'], desc="Processing Movies"):
        print(f"검색할 영화 이름: {movie_name}")  # 디버깅을 위한 로그

        # 와챠피디아 검색창 찾기
        search_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="desktop-search-field"]'))
        )
        search_field.clear()  # 이전 입력값 지우기
        search_field.send_keys(movie_name)  # 검색창에 영화 이름 입력
        search_field.send_keys(Keys.ENTER)  # Enter 키로 검색 실행
        time.sleep(2)  # 다음 작업을 위해 2초 대기

        found_movie = False
        a = len(driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[1]/section/section/div[3]/div[1]/section/section[2]/div/div[1]/div/ul/li/a'))
        
        if a >= 10 :
            a = 10
            
        for i in range(1, a+1):
            try:
                print(f"검색 결과 {i}번째 항목 확인")  # 디버깅을 위한 로그

                # 영화 링크가 존재하는지 확인 
                movie_link = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[1]/section/section/div[3]/div[1]/section/section[1]/div/div[1]/div/ul/li[{i}]/a'))
                )
                movie_link.click()
                time.sleep(1)  # 화면 전환되어 다음작업을 수행할 여유시간을 1초간 부여

                # 와챠피디아의 해당 영화 제목을 변수로 지정
                movie_title_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[1]/div[4]/div/h1'))
                )
                movie_title = movie_title_element.text.replace(" ", "").split("(")[0]  
                # 영화제목의 띄어쓰기를 없애고, '('가 있다면 그 앞쪽text만 변수로 다시 지정
                #time.sleep(1)

                # 와챠피디아의 평균별점 text를 변수로 저장 
                aver_text = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[2]/section[1]/div[2]/section[1]/div[2]/div/div[2]'))
                ).text
                #time.sleep(2)

                # 영화제목이 엑셀 데이터의 영화이름과 일치할 경우
                if movie_title == movie_name.replace(" ", ""):
                    movie_aver_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[2]/section[1]/div[2]/section[1]/div[2]/div/div[2]'))
                    )  # 와챠피디아 평균별점 xpath를 변수로 저장
                    movie_aver = movie_aver_element.text  # 평균점수를 변수로 저장
                    #time.sleep(1)

                    print(f"영화 '{movie_name}'의 평점: {movie_aver}")  # 확인용 
                    watch_aver.append(movie_aver)  # watch_aver 변수 리스트에 별점 저장.
                    found_movie = True  # 작업 완료 처리
                    break  # 포문 수행 break

                else:
                    driver.back()
                    time.sleep(2)  # 화면 전환되어 다음작업을 수행할 수 있도록 여유시간을 2초간 부여

            except Exception as inner_e:
                print(f"내부 에러 발생")
                continue

        if not found_movie:
            watch_aver.append("없음")  # watch_aver에 '없음'으로 저장

            back_to_home = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div/div[1]/header/nav/div/div/ul/li'))
            )
            back_to_home.click()  # 메인화면으로 이동              
            time.sleep(2)  # 화면 전환되어 다음작업을 수행할 수 있도록 여유시간을 2초간 부여 

except Exception as e:
    # 예외가 발생할 경우 오류 메시지 출력
    print(f"에러 발생: {e}")  # 예상 외의 문제 발생시, 식별할 수 있도록 '에러발생' 메세지 출력
    watch_aver.append('없음')  # watch_score 변수 리스트에 '없음'으로 저장
    back_to_home = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div/div[1]/header/nav/div/div/ul/li' ))  # 와챠피디아 메인화면으로 갈수 있는 xpath 주소
    )
    back_to_home.click()  # 메인화면으로 이동
    time.sleep(2)  # 화면 전환되어 다음작업을 수행할 수 있도록 여유시간을 2초간 부여
finally:
    driver.quit()  # 드라이버 종료

# 결과 저장
subset_movie_df['Watcha_Average'] = watch_aver  # watch_aver를 subset_movie_df에 추가
subset_movie_df.to_excel(f"movie_aver_{start_idx}_{end_idx}.xlsx", index=False)  # 결과를 엑셀 파일로 저장

Processing Movies:   0%|                                                                       | 0/999 [00:00<?, ?it/s]

검색할 영화 이름: 꿈꾸는 나비
검색 결과 1번째 항목 확인


Processing Movies:   0%|                                                               | 1/999 [00:03<58:34,  3.52s/it]

영화 '꿈꾸는 나비'의 평점: 평균 별점
(25명)
검색할 영화 이름: 클로버필드 10번지
검색 결과 1번째 항목 확인


Processing Movies:   0%|▏                                                              | 2/999 [00:06<57:50,  3.48s/it]

...

검색할 영화 이름: 백설공주 살인사건
검색 결과 1번째 항목 확인


Processing Movies:   4%|██▋                                                         | 44/999 [03:14<1:15:50,  4.76s/it]

영화 '백설공주 살인사건'의 평점: 평균 별점
(2.2만명)
검색할 영화 이름: 파일 : 4022일의 사육
검색 결과 1번째 항목 확인


Processing Movies:   4%|██▋                                                         | 44/999 [03:17<1:11:30,  4.49s/it]


KeyboardInterrupt: 